In [6]:
from pyswmm import Simulation
import os

i = 1
simfolder = r'C:\Users\echelsvi\git\chelsvig_urban_pesticides\probabilistic_python\input\swmm\input_' + str(i)
print(simfolder)
print(os.path.exists(simfolder))

simfile = os.path.join(simfolder, "NPlesantCreek"+"."+"inp")
print(simfile)

sim = Simulation(simfile)
sim.execute()


C:\Users\echelsvi\git\chelsvig_urban_pesticides\probabilistic_python\input\swmm\input_1
True
C:\Users\echelsvi\git\chelsvig_urban_pesticides\probabilistic_python\input\swmm\input_1\NPlesantCreek.inp

... SWMM Version 5.3.0.dev0
... Run Complete

In [3]:

import pytest_shutil, shutil, os, pandas, regex as re

# nsims
nsims = 5

# define root path for all folders
root_path = r'C:\Users\echelsvi\git\chelsvig_urban_pesticides\probabilistic_python\input\swmm'
print(os.path.exists(root_path))

# read in lhs_sampled_params
lhs_design = pandas.read_csv(
    r'C:\Users\echelsvi\git\chelsvig_urban_pesticides\probabilistic_python\io\lhs_sampled_params.csv')

# round lhs decimals
lhs_design = lhs_design.round(
    {"NImperv": 4, "NPerv": 4, "SImperv":3, "SPerv": 3, "PctZero": 3, "MaxRate": 3, "MinRate": 3, "Decay": 2, "DryTime": 0,
     "Por": 3, "WP": 3, "FC": 3, "Ksat": 3, "Roughness": 4, "Kdecay": 3, "BCoeff2": 3, "WCoeff2": 3})
print(lhs_design.head())

# do the following for each simulation...
Ite = 1
newfol = "input_" + str(Ite)
print(newfol)

newdir = os.path.join(root_path, newfol)
print(newdir)

if not os.path.exists(newdir):
    os.mkdir(newdir)
    print("Folder ", Ite, " created", "\n")
else:
    print("Folder ", Ite, "already exists")

os.getcwd()
os.chdir(newdir)

    # copy base file into new file location
old_swmm5 = os.path.join(root_path, "NPlesantCreek.inp")
new_file = os.path.join(newdir, "NPlesantCreek.inp")
shutil.copyfile(old_swmm5, new_file)

    # start reading the new file
new_swmm5 = open(new_file, "r")
filelines = new_swmm5.readlines()

True
   Unnamed: 0  NImperv   NPerv  SImperv  SPerv  PctZero  MaxRate  MinRate  \
0           0   0.0243  0.2176    1.663  2.694   40.044  119.901    0.356   
1           1   0.0152  0.3288    1.432  2.624   57.428   54.573    7.922   
2           2   0.0103  0.2024    1.411  4.672   42.669   14.351    9.796   
3           3   0.0237  0.4867    1.923  5.080    7.899   25.438    5.581   
4           4   0.0216  0.4077    1.594  2.852   57.356   34.045    2.325   

   Decay  DryTime    Por     WP     FC   Ksat  Roughness  Kdecay  BCoeff2  \
0   4.33      4.0  0.497  0.124  0.269  7.484     0.0197   0.047    1.959   
1   3.08     11.0  0.496  0.032  0.070  3.025     0.0136   0.159    1.417   
2   6.91     11.0  0.494  0.033  0.226  4.617     0.0101   0.085    1.606   
3   2.77     13.0  0.450  0.101  0.149  5.960     0.0140   0.006    1.426   
4   5.83      7.0  0.434  0.049  0.181  4.734     0.0135   0.086    0.794   

   WCoeff2  
0    0.525  
1    0.211  
2    0.860  
3    0.087  
4   

In [5]:
Num = 1  # number of conduits
row_0 = 736
Roughness = lhs_design.loc[Ite - 1, "Roughness"]
print(Roughness)

for i in range(1, Num + 1):
        row_t = row_0 + (i - 1)
        oldline = filelines[row_t]

        fixline = " ".join(oldline.split())
        listline = fixline.split()
        
        print(listline)
        print(type(listline))
        print(len(listline))
        print(listline[4])
        
        listline[4]=str(Roughness)
        print(listline)
        
        listTOstring = ' '.join([str(item) for item in listline])

        newline = listTOstring + "\n"
        filelines[row_t] = newline

print(filelines[734:736])

0.0197
['C3', 'J5', 'J4', '2554', '0.01', '0', '0', '0', '0']
<class 'list'>
9
0.01
['C3', 'J5', 'J4', '2554', '0.0197', '0', '0', '0', '0']
['C1 J1 J2 268.84 0.0197 0 0 0 0\n', 'C2 J3 J2 189 0.0197 0 0 0 0\n']


In [ ]:
     listline[4] = str(Roughness)
        listTOstring = ' '.join([str(item) for item in listline])

        newline = listTOstring + "\n"
        filelines[row_t] = newline

In [2]:
# --------------------------------------------------
# run probabilistic simulations, save output
# --------------------------------------------------

# todo run each of the probabilistic sim .inp files, and save the results to a cohesive data frame

# setup
from pyswmm import Simulation
import os, pandas
import rpy2.robjects as ro
from rpy2.robjects.packages import importr
import pandas.rpy.common as com

d = {'package.dependencies': 'package_dot_dependencies',
     'package_dependencies': 'package_uscore_dependencies'}
swmmr = importr('swmmr', robject_translations = d)

print("Hello there")



OSError: cannot load library 'C:\Program Files\R\R-3.6.1\bin\x64\R.dll': error 0x7e

In [ ]:
swmmr.read_out()

# nsims
nsims = 5

# todo set up dataframe to receive the outputs from the simulations...below

# run swmm, for each sim
for i in range(1, nsims+1):
    simfolder = r'C:\Users\echelsvi\git\chelsvig_urban_pesticides\probabilistic_python\input\swmm\input_' + str(i)
    print(simfolder)
    print(os.path.exists(simfolder))

    simfile = os.path.join(simfolder, "NPlesantCreek" + "." + "inp")
    print(simfile)

    sim = Simulation(simfile)
    sim.execute()

    # todo grab the desired output variable of interest, and store in dataframe (created above loop)

# --------------------------------------------------------
# the end
# --------------------------------------------------------